# Data and label curation

Take Code from `/Covid19/MULTIseq_ALI/preprocess_muc21058_V2.ipynb` and extend to all samples

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
import seaborn as sns

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
## Set global plotting parameters
plt.rcParams["figure.figsize"] = (6, 5)
sns.set_style("ticks")

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# sc.set_figure_params(dpi=300, frameon=False, color_map='Reds')
sc.logging.print_header()

scanpy==1.6.0 anndata==0.7.4 umap==0.4.6 numpy==1.19.1 scipy==1.5.2 pandas==1.1.1 scikit-learn==0.23.2 statsmodels==0.11.1 python-igraph==0.8.3 leidenalg==0.8.3


In [4]:
folder = "~/meshal.ansari/mnt/platte_1/Projects/201109_Covid19_ALI/"
save_dir = '/storage/groups/ml01/workspace/yuge.ji/covid-perturbation/data/'

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


BioSampleID sheet.

In [5]:
metadata = pd.read_csv('BioSample_IDs.txt', sep='\t')
metadata.set_index('identifier', inplace=True)
metadata

,patient,name,barcode_round,cells
identifier,,,,
muc21058,patient_1_2,ALI_1_Barcode_1,ALI_1,40000
muc21059,patient_1_2,ALI_1_Barcode_2,ALI_1,40000
muc21060,patient_1_2,ALI_1_Barcode_3,ALI_1,40000
muc21061,patient_1_2,ALI_1_Barcode_4,ALI_1,40000
muc21062,patient_3,ALI_2_Barcode_1,ALI_2,40000
muc21063,patient_3,ALI_2_Barcode_2,ALI_2,40000
muc21064,patient_3,ALI_2_Barcode_3,ALI_2,40000
muc21065,patient_3,ALI_2_Barcode_4,ALI_2,40000
muc22965,patient_4,ALI_3_Barcode_1,ALI_3,40000


### Read in count matrices

In [6]:
%%time
import glob
import os
dge_path = "/storage/groups/bcf/projects/SingleCell/10x/projects/schiller/201026_Covid19_ALI/cellranger"

files = []
for x in os.walk(dge_path):
    for y in glob.glob(os.path.join(x[0], '*filtered_feature_bc_matrix.h5')):
        files.append(y)

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 44.9 ms, sys: 75.8 ms, total: 121 ms
Wall time: 246 ms


In [7]:
def read_dges_10x(files, metadata):
    adatas = []
    for file in files:
        sample_id = file.split("/")[-3].lower()
        
        ## Read in dge (no filter for top barcodes in this)
        a = sc.read_10x_h5(file)  ## don’t need to transpose here
        a.var_names_make_unique() ## so it can be concatenated

        a.obs["identifier"] = sample_id
        ## Add all columns from metadata Table as Metainfo (apart from cells)
        for col in ['patient', 'name', 'barcode_round']:
            a.obs[col] = metadata.loc[sample_id, col]
            
        ## For some reason the obs names are ending with "-1"
        a.obs_names = ["%s_%s" %(sample_id, cell.replace("-1", "")) for cell in a.obs_names.values]
        adatas.append(a)
        
    return adatas

In [8]:
%%time
# Load dges
adatas = read_dges_10x(files, metadata)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.

CPU times: user 38.4 s, sys: 5.54 s, total: 43.9 s
Wall time: 43.9 s


In [9]:
%%time
## Concatenate them into one adata file (assign categories automatically)
adata = adatas[0].concatenate(adatas[1:], batch_key = "sample", join = "outer", index_unique = None)

CPU times: user 2.21 s, sys: 2.03 s, total: 4.24 s
Wall time: 4.23 s


In [10]:
## As outer join is used when concatenating the adatas, set NaN entries to zero
adata.X = np.nan_to_num(adata.X)
# adata.X = adata.X.toarray()
adata

AnnData object with n_obs × n_vars = 414497 × 20635
    obs: 'identifier', 'patient', 'name', 'barcode_round', 'sample'
    var: 'gene_ids', 'feature_types', 'genome'

Label viral genes. Viral markers sourced from: https://www.ncbi.nlm.nih.gov/nuccore/1798174254 (thanks Meshal!)

In [11]:
viral_genes = ["ORF1ab", "S", "ORF3a", "E", "M", "ORF6", "ORF7a", "ORF7b", "ORF8", "N", "ORF10",
    "ORF1ab-minus", "S-minus", "ORF3a-minus", "E-minus", "M-minus", "ORF6-minus",
    "ORF7a-minus", "ORF7b-minus", "ORF8-minus", "N-minus", "ORF10-minus"]
adata.var['viral'] = [True if gene in viral_genes else False for gene in adata.var_names]
len(viral_genes)
adata.obs["viral_counts"] = adata[:, viral_genes].X.sum(1)

Label perturbations from deMULTIplex.

In [12]:
# grab from existing adata for now
# labeled_adata.obs.to_pickle('/storage/groups/ml01/workspace/yuge.ji/covid-perturbation/data/210118_raw_obs.pkl')
obs_df = pd.read_pickle('/storage/groups/ml01/workspace/yuge.ji/covid-perturbation/data/210118_raw_obs.pkl')

In [13]:
adata.obs['treatment'] = obs_df['treatment']
adata.obs['treatment_virus'] = obs_df['treatment_virus']  # comes from deMULTIplex of barcodes
# adata.obs['infected'] = obs_df['infected']

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


How many are mislabeled? (displayed by batch)

In [16]:
for group in adata.obs['patient'].unique():
    subset = adata[adata.obs['patient'] == group]
    print(group)
    display(pd.crosstab(subset.obs['treatment_virus'].values, (subset.obs['viral_counts'].values > 0)))

patient_3


col_0,False,True
row_0,,
IFNL,2078,193
IFNL + sars-cov-2,2087,162
IFNa (b_d),2263,202
IFNa (b_d) + sars-cov-2,1446,109
IFNa_TNFa,2925,243
IFNa_TNFa + sars-cov-2,3107,251
IFNg,2678,201
IFNg + sars-cov-2,2709,199
IFNg_TGFb,1992,170


patient_4


/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


col_0,False,True
row_0,,
BB95,1517,221
BB95 + sars-cov-2,1846,314
DMSO + sars-cov-2,1992,293
Gilt,1713,243
Gilt + sars-cov-2,1708,268
IFNL,2580,373
IFNL + sars-cov-2,1637,246
IFNa (b_d),1951,279
IFNa (b_d) + sars-cov-2,2182,348


patient_1_2


/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


col_0,False,True
row_0,,
IFNa (b_d),4616,522
IFNa (b_d) + sars-cov-2,4105,481
IFNa_TNFa,1770,187
IFNa_TNFa + sars-cov-2,1634,209
IFNg,4279,526
IFNg + sars-cov-2,2670,323
Sorafenib,1496,199
Sorafenib + sars-cov-2,1645,219
TGFb2,2749,375


In [17]:
%%time
loc = save_dir + '210118_curated'
sc.write(loc, adata)
print('saved to', loc)

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])


saved to /storage/groups/ml01/workspace/yuge.ji/covid-perturbation/data/210118_curated
CPU times: user 2min 30s, sys: 3.25 s, total: 2min 33s
Wall time: 2min 47s
